# Искусственный интеллект 2021. Demo ML. Языки

## **A. Языки**

Вам дан датасет с предложениями на разных языках Европы: по 15000 предложений на каждом из 20 языков из списка: bg,cs,da,de,el,es,et,ﬁ,fr,hu,it,lt,lv,nl,pl,pt,ro,sk,sl,sv.

**Ваша цель:** научиться распознавать по слову его язык.

Метрика качества **accuracy** — доля слов, у которых правильно определен язык среди всех слов.

In [1]:
# Позволяет монтировать гугл-диск в colab
# и получить доступ к папкам и файлам на гугл-диске
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Модуль **os** из стандартной библиотеки языка программирования Python обычно используется для работы с установленной ОС, а также файловой системой ПК. Он содержит массу полезных методов для взаимодействия с файлами и папками на жестком диске. Программы, работающие с модулем os, не зависят от типа ОС и являются легко переносимыми на другую платформу.

In [2]:
import re
import pandas as pd
import os

url = os.chdir(r"./gdrive/MyDrive/Я-профи подготовка по машинному обучению/Языки/data kaggle")
os.getcwd()

'/content/gdrive/MyDrive/Я-профи подготовка по машинному обучению/Языки/data kaggle'

# Создание и загрузка данных

In [3]:
os.getcwd()

'/content/gdrive/MyDrive/Я-профи подготовка по машинному обучению/Языки/data kaggle'

In [4]:
files = os.listdir(url)
files

['test_submission',
 'train_de',
 'test_data.csv',
 'train_da',
 'train_cs',
 'train_bg',
 'train_el',
 'sample_submit (1).csv',
 'test',
 'train_hu',
 'train_lv',
 'train_et',
 'train_fr',
 'train_lt',
 'train_fi',
 'train_es',
 'train_it',
 'train_nl',
 'train_pt',
 'train_sv',
 'train_sl',
 'train_pl',
 'train_ro',
 'train_sk',
 'submition_adelya.csv']

In [5]:
files = ['train_de', 'train_da', 'train_cs', 'train_bg', 'train_el', 'train_hu', 'train_lv',
         'train_et', 'train_fr', 'train_lt', 'train_fi', 'train_es', 'train_it', 'train_nl',
         'train_pt', 'train_sv', 'train_sl', 'train_pl', 'train_ro', 'train_sk']
len(files)

20

Так как у нас 20 файлов в которых находятся предложения на 20 различных языках, в первую очередь нужно как то объедининть все данные и создать размеченную выборку для обучения.

In [ ]:
lang = {'train_da':[], 'train_bg':[], 'train_cs':[], 'train_de':[], 'train_el':[], 'train_es':[], 'train_et':[],
        'train_fi':[], 'train_fr':[], 'train_hu':[], 'train_it':[], 'train_lt':[], 'train_lv':[], 'train_nl':[],
        'train_pl':[], 'train_pt':[], 'train_ro':[], 'train_sk':[], 'train_sl':[], 'train_sv':[]}

for file in files:
    with open(file, "r", encoding='utf-8') as f:
        name = file
        k = []
        # считываем все строки одного файла и сохраняем их во временный список k
        for line in f.readlines():
            k.append(line)
        lang[name] = k

In [8]:
# re - библиотека для работы с регулярными выражениями и строками
for l in lang:
    if re.findall('train', l):
        k = l.split('train_')[1]
        print(l, '|', k)

train_da | da
train_bg | bg
train_cs | cs
train_de | de
train_el | el
train_es | es
train_et | et
train_fi | fi
train_fr | fr
train_hu | hu
train_it | it
train_lt | lt
train_lv | lv
train_nl | nl
train_pl | pl
train_pt | pt
train_ro | ro
train_sk | sk
train_sl | sl
train_sv | sv


In [9]:
train = pd.DataFrame(columns=['text', 'language'])

for l in lang:
    if re.findall('train', l):
        tmp = pd.DataFrame()
        tmp['text'] = lang[l]
        tmp['language'] = l.split('train_')[1]
        train = pd.concat([train, tmp], ignore_index=True)

train

text language
0       Gennem pressen og tv vil De være bekendt med e...       da
1       Det er netop dér, De - hvis De ønsker det - ka...       da
2       På anmodning af et fransk parlamentsmedlem, hr.\n       da
3       Men, fru formand, det, som jeg havde anmodet o...       da
4       Fru Plooij-van Gorsel, jeg kan oplyse Dem om, ...       da
...                                                   ...      ...
299995  Det program som har lagts fram i dag tyder i a...       sv
299996  I synnerhet gläder jag mig åt att frågorna om ...       sv
299997  Jag skulle vilja koncentrera mig på en av huvu...       sv
299998                        Men det behövs mer än så.\n       sv
299999           Toppmötet i Feira, mitt sista exempel.\n       sv

[300000 rows x 2 columns]

In [10]:
print(f'What languages are represented in the data: {train["language"].unique()}')
print(f'Ltn languages are represented in the data: {len(train["language"].unique())}')
print('-'*100)
print(f'How many sentences are presented for each language: \n{train["language"].value_counts()}')

What languages are represented in the data: ['da' 'bg' 'cs' 'de' 'el' 'es' 'et' 'fi' 'fr' 'hu' 'it' 'lt' 'lv' 'nl'
 'pl' 'pt' 'ro' 'sk' 'sl' 'sv']
Ltn languages are represented in the data: 20
----------------------------------------------------------------------------------------------------
How many sentences are presented for each language: 
fi    15000
pl    15000
el    15000
sl    15000
hu    15000
bg    15000
de    15000
nl    15000
sk    15000
da    15000
ro    15000
sv    15000
cs    15000
et    15000
lt    15000
lv    15000
es    15000
pt    15000
fr    15000
it    15000
Name: language, dtype: int64


In [11]:
# Обработка и загрузка тестового набора данных
test = pd.read_csv('test_data.csv')
test.drop(columns=['index'])
test

index            text
0         0    szállítanak 
1         1       pracovni 
2         2        fašisti 
3         3         passer 
4         4  culpabilização
...     ...             ...
8515   8515        middags 
8516   8516      tunnetega 
8517   8517        tocassem
8518   8518          urale 
8519   8519  labialisassiez

[8520 rows x 2 columns]

# Обработка текста

Любой рабочий процесс анализа текстовых данных начинается с их предварительной обработки и очистки текста. 

**Рассмотрим стандартный конвейер (pipeline) предобработки:**

- удаление знаков препинания, лишних и спец символов
- приведение к одному регистру
- токенизировать текст – разбить текст на предложения, слова и другие единицы;
- удалить стоп-слова;
- привести слова к нормальной форме;
- векторизовать тексты – сделать числовые представления текстов для их дальнейшей обработки классификатором.

Все эти шаги служат для уменьшения шума, присущего любому обычному тексту, и повышения точности результатов классификатора. Для решения указанных задач есть несколько отличных библиотек, например, NLTK, TextBlob и spaCy, pymorphy.

**Посмотрев на разные предложения и исходя из задачи можно сделать следующие выводы:**
- знаки препенания не зависят от языка (точка что в английском, что в русском) - значит можно удалить все знаки препинания
- стоп-слова могут иметь значения в данной задаче

In [12]:
import nltk
# nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
# russian_stopwords = stopwords.words("russian")

def remove_punct(text):
    # удаление пунктуации в тексте
    table = {33: ' ', 34: ' ', 35: ' ', 36: ' ', 37: ' ', 38: ' ', 39: ' ', 40: ' ', 41: ' ', 42: ' ',
             43: ' ', 44: ' ', 45: ' ', 46: ' ', 47: ' ', 58: ' ', 59: ' ', 60: ' ', 61: ' ', 62: ' ',
             63: ' ', 64: ' ', 91: ' ', 92: ' ', 93: ' ', 94: ' ', 95: ' ', 96: ' ', 123: ' ', 124: ' ', 125: ' ', 126: ' '}
    return text.translate(table)

def txt_prep(df):
    # функция приводит весь текст к нижнему регистру
    # удаляет пунктуацию
    df['initial_text'] = df['text']
    df['text'] = df['text'].str.lower() # Hello - hello
    df['text'] = df['text'].map(lambda x: remove_punct(x))
    df['text'] = df['text'].str.replace('»', '') 
    df['text'] = df['text'].str.replace('«', '') 
    df['text'] = df['text'].str.replace('"', '')
    df['text'] = df['text'].str.replace('\n', '')
    df['text'] = df['text'].str.replace(r"\d+", "", flags=re.UNICODE)

    return df

train = txt_prep(train)
train

text  ...                                       initial_text
0       gennem pressen og tv vil de være bekendt med e...  ...  Gennem pressen og tv vil De være bekendt med e...
1       det er netop dér  de   hvis de ønsker det   ka...  ...  Det er netop dér, De - hvis De ønsker det - ka...
2         på anmodning af et fransk parlamentsmedlem  hr   ...  På anmodning af et fransk parlamentsmedlem, hr.\n
3       men  fru formand  det  som jeg havde anmodet o...  ...  Men, fru formand, det, som jeg havde anmodet o...
4       fru plooij van gorsel  jeg kan oplyse dem om  ...  ...  Fru Plooij-van Gorsel, jeg kan oplyse Dem om, ...
...                                                   ...  ...                                                ...
299995  det program som har lagts fram i dag tyder i a...  ...  Det program som har lagts fram i dag tyder i a...
299996  i synnerhet gläder jag mig åt att frågorna om ...  ...  I synnerhet gläder jag mig åt att frågorna om ...
299997  jag skulle vilja koncentrera mig på en av huvu...  ...  Jag skulle vilja koncentrera mig på en av huvu...
299998                          men det behövs mer än så   ...                        Men det behövs mer än så.\n
299999             toppmötet i feira  mitt sista exempel   ...           Toppmötet i Feira, mitt sista exempel.\n

[300000 rows x 3 columns]

In [13]:
train.loc[26, 'initial_text']

'Jeg håber, at der tages højde for mit forslag ved afstemningen i morgen.\n'

In [14]:
train.loc[26, 'text']

'jeg håber  at der tages højde for mit forslag ved afstemningen i morgen '

В данной задаче мы решаем с Вами задачу мультиклассовой классификации (20 языков - 20 классов).
Но все классы представлены в виде категориальной переменной - закодируем ее с помощью `LabelEncoder()`.

При этом необходимо сохранить словарь для обратного преобразования.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

label_encoder.fit(train['language']) 
train['language'] = label_encoder.transform(train['language']) 

mapping = dict(zip(label_encoder.classes_, range(len(label_encoder.classes_))))

mapping

{'bg': 0,
 'cs': 1,
 'da': 2,
 'de': 3,
 'el': 4,
 'es': 5,
 'et': 6,
 'fi': 7,
 'fr': 8,
 'hu': 9,
 'it': 10,
 'lt': 11,
 'lv': 12,
 'nl': 13,
 'pl': 14,
 'pt': 15,
 'ro': 16,
 'sk': 17,
 'sl': 18,
 'sv': 19}

In [16]:
# mapping
inverse_dict = dict([val,key] for key,val in mapping.items())
inverse_dict

{0: 'bg',
 1: 'cs',
 2: 'da',
 3: 'de',
 4: 'el',
 5: 'es',
 6: 'et',
 7: 'fi',
 8: 'fr',
 9: 'hu',
 10: 'it',
 11: 'lt',
 12: 'lv',
 13: 'nl',
 14: 'pl',
 15: 'pt',
 16: 'ro',
 17: 'sk',
 18: 'sl',
 19: 'sv'}

# Обучение и подбор модели

In [20]:
df_small = train.sample(frac=0.01)

X_small = df_small.drop(['language','initial_text'],axis=1)
y_small = df_small['language']

df_small.shape

(3000, 3)

In [22]:
df_small['language'].value_counts()

1     174
19    164
2     163
16    163
18    157
11    156
9     154
13    153
6     153
3     150
15    150
0     150
8     149
17    146
14    144
12    142
4     141
7     135
10    132
5     124
Name: language, dtype: int64

In [30]:
X_small

text
213852  jestem przekonana  że podjęliśmy już pewne waż...
216894  oprócz niego zatrzymana została również olga k...
259103  okrem toho dnes ráno sa uskutočnilo trojstrann...
18171            не разполагаме с достатъчно детски ясли 
94855              meil on tasakaal põhja ja lõuna vahel 
...                                                   ...
289738  vi får inte glömma att belgien och italien har...
48445   die kommission von herrn prodi trägt keine sch...
28782   мащабът на бедствието допълнително се изостря ...
43678   tohoto výsledku bylo dosaženo po několika dlou...
100642  pidades silmas kõnealust suundumust ja eli ees...

[3000 rows x 1 columns]

In [31]:
y_small

213852    14
216894    14
259103    17
18171      0
94855      6
          ..
289738    19
48445      3
28782      0
43678      1
100642     6
Name: language, Length: 3000, dtype: int64

**Векторизация текста**

Text representation
We have various options:

- Count Vectors as features
- TF-IDF Vectors as features
- Word2Vec
- FastText

Полезные сслыки:

https://habr.com/ru/company/ods/blog/329410/

https://dataaspirant.com/word-embedding-techniques-nlp/


In [ ]:
# # Count Vectors as features
# 0 - 'мама мыла раму'
# 1 - 'петя пошел гулять и пошел в магазин'

#     мама | мыла | раму | петя | пошел | гулять | и | в | магазин
# 0     1     1      1      0      0        0      0   0      0
# 1     0     0      0      1      2        1      1   1      1

In [ ]:
# 1. Словные n-gram

# Мама мыла раму - 2-gram
# мама мыла   |   мыла раму

# 2. Символьные n-gram 

# Радуга - 2-gram:
# ра      ад      ду      уг      га

In [32]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_small['text'], y_small, test_size=0.33, random_state=42)

vectorizer = CountVectorizer(ngram_range=(3,6), analyzer='char', max_features = 25000)

X_train_vector = vectorizer.fit_transform(X_train).toarray()
X_test_vector = vectorizer.transform(X_test).toarray()

In [33]:
X_train_vector.shape

(2010, 25000)

In [35]:
X_test_vector.shape

(990, 25000)

In [34]:
print(f'X train shape: {X_train.shape}')
print(f'X test shape: {X_test.shape}')

X train shape: (2010,)
X test shape: (990,)


In [36]:
# чисто для наглядности как работает CountVectorizer
count_vect_df = pd.DataFrame(X_train_vector, columns=vectorizer.get_feature_names())
count_vect_df

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


a    a a   a ab   a b   a bi   a c  ...  яване  ят   ята  ята   ято  ято 
0       0     0      0     0      0     0  ...      0    0    0     0    0     0
1       0     0      0     0      0     0  ...      0    0    0     0    0     0
2       0     0      0     0      0     0  ...      0    0    0     0    0     0
3       0     0      0     0      0     0  ...      0    0    0     0    0     0
4       0     0      0     0      0     0  ...      0    0    0     0    0     0
...   ...   ...    ...   ...    ...   ...  ...    ...  ...  ...   ...  ...   ...
2005    0     0      0     0      0     0  ...      0    0    0     0    0     0
2006    1     1      0     0      0     0  ...      0    0    0     0    0     0
2007    0     0      0     0      0     0  ...      0    0    0     0    0     0
2008    0     0      0     0      0     0  ...      0    0    0     0    0     0
2009    0     0      0     0      0     0  ...      0    0    0     0    0     0

[2010 rows x 25000 columns]

In [38]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.3 MB/s 


In [39]:
%%time
from sklearn.metrics import classification_report #building a price prediction
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix, accuracy_score

model = CatBoostClassifier(silent = True)

model.fit(X_train_vector, y_train)
y_pred_test = model.predict(X_test_vector)

print(accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test))

0.9636363636363636
              precision    recall  f1-score   support

           0       0.98      1.00      0.99        57
           1       1.00      0.92      0.96        62
           2       0.96      0.92      0.94        49
           3       1.00      1.00      1.00        56
           4       1.00      1.00      1.00        51
           5       0.97      0.92      0.94        36
           6       0.87      0.98      0.92        54
           7       0.98      1.00      0.99        43
           8       0.98      0.97      0.97        58
           9       0.95      0.95      0.95        40
          10       1.00      0.97      0.99        39
          11       0.98      0.98      0.98        47
          12       0.93      0.96      0.94        53
          13       0.98      0.96      0.97        48
          14       0.96      0.96      0.96        56
          15       1.00      0.95      0.98        43
          16       0.98      0.98      0.98        53
        

In [34]:
%%time
# xgb, gradientboosting, svc
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=1000)

clf.fit(X_train_vector, y_train)

y_pred = clf.predict(X_test_vector)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       118
           1       0.91      0.97      0.94       118
           2       0.97      0.99      0.98       126
           3       0.99      1.00      1.00       102
           4       1.00      1.00      1.00       125
           5       0.98      0.93      0.96       122
           6       0.96      0.99      0.98       109
           7       0.98      0.98      0.98       123
           8       1.00      0.97      0.99       106
           9       0.97      1.00      0.98       125
          10       1.00      0.98      0.99       108
          11       0.98      0.96      0.97       116
          12       0.93      0.99      0.96        99
          13       1.00      0.98      0.99       111
          14       0.98      0.98      0.98       118
          15       0.92      0.97      0.95       113
          16       0.99      0.98      0.99       110
          17       0.96    

In [40]:
test

index            text
0         0    szállítanak 
1         1       pracovni 
2         2        fašisti 
3         3         passer 
4         4  culpabilização
...     ...             ...
8515   8515        middags 
8516   8516      tunnetega 
8517   8517        tocassem
8518   8518          urale 
8519   8519  labialisassiez

[8520 rows x 2 columns]

In [41]:
%%time
test_vector = vectorizer.transform(test['text']).toarray()

y_pred_lang = model.predict(test_vector)
y_pred_lang #0, 4, 1 ...

CPU times: user 1min 49s, sys: 2.66 s, total: 1min 52s
Wall time: 1min 51s


In [42]:
y_pred_lang

array([[ 9],
       [ 0],
       [11],
       ...,
       [ 0],
       [ 0],
       [ 0]])

In [37]:
# mapping
inverse_dict = dict([val,key] for key,val in mapping.items())
inverse_dict

{0: 'bg',
 1: 'cs',
 2: 'da',
 3: 'de',
 4: 'el',
 5: 'es',
 6: 'et',
 7: 'fi',
 8: 'fr',
 9: 'hu',
 10: 'it',
 11: 'lt',
 12: 'lv',
 13: 'nl',
 14: 'pl',
 15: 'pt',
 16: 'ro',
 17: 'sk',
 18: 'sl',
 19: 'sv'}

In [43]:
# Формуется ответ ответ в нужном виде
test_pred = pd.DataFrame(y_pred_lang, columns=['lang_samp'])
test_pred.reset_index(inplace=True)

test_pred['lang_samp'] = test_pred['lang_samp'].map(inverse_dict).fillna(test_pred['lang_samp'])
test_pred

index lang_samp
0         0        hu
1         1        bg
2         2        lt
3         3        bg
4         4        bg
...     ...       ...
8515   8515        bg
8516   8516        bg
8517   8517        bg
8518   8518        bg
8519   8519        bg

[8520 rows x 2 columns]

In [44]:
test_pred['lang_samp'].value_counts()

bg    6209
el     461
et     442
fi     351
lt     259
lv     207
pl     149
hu     146
cs     120
sl      81
sv      31
ro      21
sk      17
nl       9
fr       8
da       4
it       4
de       1
Name: lang_samp, dtype: int64

In [45]:
test_pred.to_csv('submition_adelya.csv', index=False)

In [46]:
from google.colab import files
files.download("submition_adelya.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>